# 🚀 DERL-LEM: Zero to Hero Implementation Guide

This comprehensive guide will walk you through the implementation and usage of the **Decentralized Energy Reinforcement Learning for Local Energy Markets (DERL-LEM)** framework. We'll start with basic concepts and simple examples, then gradually introduce more complex features.

## 📋 Table of Contents

1. [Introduction](#1-introduction)
2. [Installation and Setup](#2-installation-and-setup)
3. [Basic Simulation](#3-basic-simulation)
4. [Zero to Hero: Complete Example](#4-zero-to-hero-complete-example)
5. [Working with Agents](#5-working-with-agents)
6. [Market Mechanisms](#6-market-mechanisms)
7. [Coordination Models](#7-coordination-models)
8. [Analysis and Evaluation](#8-analysis-and-evaluation)
9. [Advanced Scenarios](#9-advanced-scenarios)
10. [Preference-Based Matching](#10-preference-based-matching)
11. [Advanced Scientific Analysis](#11-advanced-scientific-analysis)
12. [Quickstart Guide](#12-quickstart-guide)
13. [Full Pipeline Examples](#13-full-pipeline-examples)
14. [Extending the Codebase](#14-extending-the-codebase)
15. [Troubleshooting](#15-troubleshooting)

---

## 1. 📖 Introduction

The DERL-LEM framework is a simulation environment for studying decentralized multi-agent reinforcement learning in local energy markets.


### 🎯 What You Can Do

The framework allows researchers to:

- ✅ **Simulate interactions** between agents with distributed energy resources (DERs)
- ✅ **Test different market clearing mechanisms** and compare their effectiveness
- ✅ **Implement and evaluate implicit coordination strategies** for decentralized systems
- ✅ **Analyze economic, grid stability, and resource utilization metrics**
- ✅ **Verify the effectiveness of decentralized approaches** compared to centralized systems

### 💡 What are Local Energy Markets?

Local Energy Markets (LEMs) enable neighbors to trade energy directly with each other. Unlike traditional centralized power systems where consumers buy electricity from utility companies, LEMs allow:

- 🏠 **Homeowners with solar panels** to sell excess electricity to neighbors
- 🔋 **Battery owners** to store energy when cheap and sell when prices rise
- 👥 **Consumers** to purchase locally-generated renewable energy

### 🤔 Why Decentralized Reinforcement Learning?

**Traditional energy markets** rely on centralized control, which can:
- ❌ Create single points of failure
- ❌ Struggle with privacy concerns
- ❌ Face scalability challenges in highly distributed systems

**Reinforcement learning** allows agents to learn optimal strategies through experience, while **decentralized approaches** enable:
- ✅ Increased resilience through removing single points of failure
- ✅ Enhanced privacy by keeping sensitive data local
- ✅ Better scalability by distributing computational load


## 2. 🛠️ Installation and Setup

### ✅ Prerequisites

- **Python 3.11+**: Required for modern Python features
- **Miniforge**: Recommended for environment management (or use Conda)

### 📦 Key Dependencies

- **PyTorch** 🧠: Deep learning framework for agent neural networks
- **Gymnasium** 🎮: Environment interface for reinforcement learning
- **NumPy** 🔢: Numerical computing for data manipulation
- **Matplotlib** 📊: Visualization of results and metrics
- **Pandas** 📈: Data analysis and manipulation

### 📝 Setup Steps

**1. Clone the repository:**

```bash
git clone https://github.com/yourusername/derl-lem.git
cd derl-lem
```

**2. Create and activate a conda environment:**

```bash
conda env create -n derl-lem -f environment.yml
conda activate derl-lem
```

**3. Install dependencies:**

```bash
conda env update -f environment.yml --prune
```


In [ ]:
# Verify installation
try:
    from src.environment.lem import LocalEnergyMarket
    print("✅ Installation successful!")
    print("🚀 Ready to start using DERL-LEM!")
except ImportError as e:
    print(f"❌ Installation issue: {e}")
    print("📝 Please check:")
    print("  - Python version is 3.11+")
    print("  - All dependencies are installed")
    print("  - Environment is activated")


## 3. 🎮 Basic Simulation

Let's start with a minimal example to understand the core components. We'll create a simple local energy market with a few agents and run a basic simulation.

### 🔑 Understanding Key Concepts

Before diving into code, let's understand the basic components:

- **Environment** 🌍: The simulation environment representing the energy market
- **Agents** 👥: Entities that can buy/sell energy (households, businesses)
- **Actions** 🎯: Decisions agents make (price, quantity, buy/sell)
- **Observations** 👁️: Information agents receive about the market state
- **Rewards** 💰: Feedback agents receive based on their actions


### 🏗️ Creating a Basic Environment

The code below demonstrates how to:
1. Create a market configuration defining prices and intervals
2. Create an environment configuration defining agents and simulation length
3. Initialize the market environment
4. Reset the environment to start a fresh simulation


In [ ]:
# Add parent directory to path to import src modules
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

import numpy as np
from src.market.matching import MarketConfig, ClearingMechanism
from src.environment.lem import LocalEnergyMarket

# 1. Configure the environment
market_config = MarketConfig(
    min_price=0.05,
    max_price=0.5,
    min_quantity=0.0,
    max_quantity=100.0,
    price_mechanism=ClearingMechanism.AVERAGE,
    blockchain_difficulty=2,
    visualize_blockchain=False,
    enable_partner_preference=True
)

env_config = {
    "max_steps": 96,    # 96 steps = 15-minute intervals for 24 hours
    "num_agents": 5,    # Start with 5 agents (households/businesses)
    "seed": 42,         # For reproducibility of results
    "market_config": market_config
}

# 2. Create the environment
env = LocalEnergyMarket(env_config=env_config)

# 3. Reset the environment to initialize
observations, info = env.reset(seed=42)

# Let's examine what we've created
print("✅ Environment initialized!")
print(f"📊 Number of agents: {len(env.agents)}")
print(f"👁️ Observation space example: {env.observation_space[list(env.agents)[0]]}")
print(f"🎯 Action space example: {env.action_space[list(env.agents)[0]]}")
print(f"🔑 Initial observations keys: {list(observations.keys())}")


### 🏃 Running a Simple Simulation Step

Let's run a single step of the simulation to see how it works:

**What's happening here?**

1. Each agent submits an "order" to the market with:
   - A price they're willing to buy/sell at
   - A quantity of energy to trade
   - Whether they want to buy or sell
2. The market matches buy and sell orders based on the market mechanism
3. Agents receive rewards based on successful trades (profit from selling or utility from buying)


In [ ]:
# Create random actions for all agents
actions = {}
for agent_id in observations.keys():
    # Actions represent:
    # - Price: The price per kWh the agent is willing to buy/sell at
    # - Quantity: Amount of energy in kWh the agent wants to trade
    # - is_buy: 1 = agent wants to buy energy, 0 = agent wants to sell energy
    actions[agent_id] = env.action_space[agent_id].sample()

# Execute one step in the environment
next_observations, rewards, terminated, truncated, info = env.step(actions)

# Examining the results
print("\n📊 Step Results:")
print(f"💰 Market clearing price: ${info.get('market_price', 'N/A'):.4f} per kWh")
print(f"⚡ Market volume: {info.get('market_volume', 'N/A'):.2f} kWh")
print(f"🤝 Number of successful trades: {len(info.get('trades', []))}")
print("\n💵 Rewards by agent (profit/loss in $):")
for agent_id, reward in rewards.items():
    print(f"  {agent_id}: ${reward:.4f}")


### 📈 Creating a Complete Basic Simulation

Now let's put it all together for a full 24-hour simulation with visualization:


In [ ]:
import matplotlib.pyplot as plt
from pathlib import Path

# Setup output directory for saving results
output_dir = Path("./results/basic_simulation")
output_dir.mkdir(parents=True, exist_ok=True)

# 1. Configure the environment (reusing previous setup)
market_config = MarketConfig(min_price=0.05, max_price=0.5, min_quantity=0.0, max_quantity=100.0)
env_config = {
    "max_steps": 96,
    "num_agents": 5,
    "seed": 42,
    "market_config": market_config
}
env = LocalEnergyMarket(env_config=env_config)
observations, info = env.reset(seed=42)

# 2. Run the simulation
market_prices = []  # Track market clearing prices
market_volumes = []  # Track trading volumes
total_rewards = []  # Track total rewards across all agents
agent_rewards = {agent_id: [] for agent_id in observations.keys()}  # Track individual agent rewards

for step in range(env_config["max_steps"]):
    # Generate random actions for all agents
    actions = {}
    for agent_id in observations.keys():
        actions[agent_id] = env.action_space[agent_id].sample()

    # Execute step
    observations, rewards, terminated, truncated, info = env.step(actions)

    # Record data
    market_prices.append(info.get("market_price", 0))
    market_volumes.append(info.get("market_volume", 0))
    total_rewards.append(sum(rewards.values()))

    # Record individual agent rewards
    for agent_id, reward in rewards.items():
        agent_rewards[agent_id].append(reward)

    # Check if all agents are done
    if all(terminated.values()):
        print(f"Simulation completed early at step {step}")
        break

# Convert time steps to hours for plotting
hours = [step * 0.25 for step in range(len(market_prices))]  # 15-min = 0.25 hours

# 3. Visualize results
plt.figure(figsize=(12, 10))

# Plot market prices
plt.subplot(3, 1, 1)
plt.plot(hours, market_prices)
plt.title('Market Clearing Prices Over Time')
plt.ylabel('Price ($/kWh)')
plt.grid(True)

# Plot market volumes
plt.subplot(3, 1, 2)
plt.plot(hours, market_volumes)
plt.title('Trading Volumes Over Time')
plt.ylabel('Volume (kWh)')
plt.grid(True)

# Plot total rewards
plt.subplot(3, 1, 3)
plt.plot(hours, total_rewards)
plt.title('Total Market Rewards')
plt.xlabel('Hour of Day')
plt.ylabel('Reward ($)')
plt.grid(True)

plt.tight_layout()
plt.savefig(output_dir / "basic_simulation_results.png")
print(f"✅ Results saved to {output_dir / 'basic_simulation_results.png'}")

# Print summary
print(f"\n📊 Final market statistics:")
print(f"  💰 Average price: ${np.mean(market_prices):.4f} per kWh")
print(f"  ⚡ Total trading volume: {np.sum(market_volumes):.2f} kWh")
print(f"  🏆 Most profitable agent: {max(agent_rewards.items(), key=lambda x: sum(x[1]))[0]}")


## 4. 🎯 Zero to Hero: Complete Example

This end-to-end example walks you through the complete workflow:

1. **Basic simulation setup and agent-environment interaction**
2. **Metrics calculation and visualization**
3. **Coordination and implicit cooperation**
4. **Multi-agent reinforcement learning (MARL) training**
5. **Evaluation and reporting**

---

### 4.1. Basic Simulation: Environment & Agents


In [ ]:
# Example: Basic simulation setup
# Note: This is a simplified example. Adjust imports based on your actual codebase structure.

# from src.environment.market_env import LocalEnergyMarket
# from src.config.settings import EnvironmentConfig, MarketConfig, AgentConfig, DERConfig
# from src.agents.der_agent import DERAgent, PricingStrategy

# Set up environment config
# env_config = EnvironmentConfig(max_steps=24, num_agents=5, market=MarketConfig())
# env = LocalEnergyMarket(config=env_config)
# observations, info = env.reset(seed=42)

# Create agents
# agents = []
# for i in range(env_config.num_agents):
#     agent = DERAgent(
#         agent_id=f"agent_{i}",
#         agent_config=AgentConfig(),
#         der_config=DERConfig(),
#         observation_space=env.observation_space[f"agent_{i}"],
#         action_space=env.action_space,
#         pricing_strategy=PricingStrategy.ADAPTIVE
#     )
#     agents.append(agent)

# Run a simple simulation
# for step in range(env_config.max_steps):
#     actions = {agent.agent_id: env.action_space.sample() for agent in agents}
#     next_observations, rewards, dones, info = env.step(actions)
#     # (Optional) print or log step info

print("📝 This section demonstrates how to set up agents and run simulations.")
print("💡 Uncomment and adapt the code above based on your codebase structure.")


### 4.2. Metrics Calculation & Visualization

Track and visualize market performance metrics:


In [ ]:
# Example: Metrics calculation and visualization
# prices, volumes, total_rewards = [], [], []
# for step in range(env_config.max_steps):
#     actions = {agent.agent_id: env.action_space.sample() for agent in agents}
#     next_observations, rewards, dones, info = env.step(actions)
#     prices.append(info.get("market_price", 0.0))
#     volumes.append(info.get("market_volume", 0.0))
#     total_rewards.append(sum(rewards.values()))

# plt.figure(figsize=(10, 6))
# plt.subplot(2,1,1)
# plt.plot(prices, label='Market Price')
# plt.legend()
# plt.subplot(2,1,2)
# plt.plot(np.cumsum(total_rewards), label='Cumulative Rewards')
# plt.legend()
# plt.tight_layout()
# plt.show()

print("📊 This section shows how to collect and visualize metrics.")


## 5. 👥 Working with Agents

Now let's explore how to work with different types of agents and their configurations. In real energy markets, different participants have different capabilities and strategies.

### 🔍 Understanding Agent Types

In local energy markets, there are three main types of participants:

| Agent Type | Description                | Examples                                            |
| ---------- | -------------------------- | --------------------------------------------------- |
| **Producer** | Generates energy to sell   | Solar farms, wind turbines                          |
| **Consumer** | Purchases energy to use    | Regular households, businesses                      |
| **Prosumer** | Both produces and consumes | Homes with solar panels, businesses with generators |


### 🔋 Understanding the Battery Model

Batteries are crucial in modern energy systems as they:

- ✅ **Store excess renewable generation** for later use
- 💰 **Enable buying energy when cheap** and using it when expensive
- 🔌 **Provide backup power** during outages
- ⚖️ **Help stabilize the grid** by balancing supply and demand

**Key Battery Parameters:**
- **Nominal Capacity**: Total energy storage capacity (kWh)
- **SOC Range**: State of charge operating range (min/max)
- **Efficiency**: Charge/discharge efficiency (typically 85-95%)
- **Self-discharge**: Energy loss when idle


In [ ]:
# Example: Battery model usage
# from src.agent.battery import Battery
# from src.profile.der import DERProfileHandler

# Create battery storage system
# battery = Battery(
#     nominal_capacity=50.0,         # 50 kWh battery
#     min_soc=0.2,                   # Minimum 20% state of charge
#     max_soc=0.9,                   # Maximum 90% state of charge
#     charge_efficiency=0.95,         # 95% charge efficiency
#     discharge_efficiency=0.95       # 95% discharge efficiency
# )

# Initialize battery with 50% state of charge
# battery.initialize(soc=0.5)
# print(f"Battery initialized at {battery.get_state()['soc']*100:.1f}% charge")

# Simulate battery operation over 24 hours
# Typical daily usage pattern:
# - Morning (6-9 AM): Discharge during breakfast/morning activities
# - Midday (10 AM-2 PM): Charge from solar production
# - Evening (5-8 PM): Discharge during dinner/evening activities

print("🔋 This section demonstrates battery modeling.")
print("💡 Batteries enable time-shifting of energy for better coordination.")


### 💰 Understanding Pricing Strategies

In energy markets, pricing strategies determine how agents set their buying and selling prices. The framework supports several strategies:

| Strategy          | Description                        | Real-world Example            |
| ----------------- | ---------------------------------- | ----------------------------- |
| **Fixed**             | Uses a constant price              | Utility flat-rate plans       |
| **Adaptive**          | Adjusts based on market conditions | Dynamic retail pricing        |
| **Competitive**       | Aims to undercut competitors       | Competitive energy retailers  |
| **Time Varying**      | Changes price based on time of day | Time-of-use electricity rates |
| **Demand Responsive** | Adjusts price based on demand      | Real-time pricing programs    |


## 6. 💼 Market Mechanisms

Now, let's explore the different market mechanisms available in the framework. Each mechanism has different properties that affect how trades are matched and priced.

### 🔍 Understanding Market Mechanisms

Market mechanisms determine how buy and sell orders are matched and how prices are calculated. Different mechanisms have different strengths:

| Mechanism                 | Description                          | Key Properties              | Best For          |
| ------------------------- | ------------------------------------ | --------------------------- | ----------------- |
| **Pay-As-Bid**                | Each trade executes at the bid price | Simple and intuitive        | New markets       |
| **Vickrey Auction**           | Second-price auction                 | Encourages truthful bidding | Research settings |
| **Continuous Double Auction** | Immediate matching                   | High liquidity              | Active markets    |
| **Spatial Average Price**     | Considers physical grid              | Respects grid constraints   | Physical networks |


In [ ]:
# Example: Comparing market mechanisms
# This demonstrates how different mechanisms affect market outcomes

print("📊 Market Mechanism Comparison:")
print("=" * 60)
print("1. Pay-As-Bid: Simple, trades execute at bid prices")
print("2. Vickrey Auction: Second-price auction mechanism")
print("3. Continuous Double Auction: Continuous price discovery")
print("4. Spatial Average Price: Considers grid constraints")
print("\n💡 Each mechanism has different properties:")
print("   - Price discovery efficiency")
print("   - Trading volume")
print("   - Grid constraint handling")
print("   - Computational complexity")


## 7. 🤝 Coordination Models

Now, let's explore how to work with the coordination models for implicit cooperation.

### 📡 Creating and Using Coordination Signals

Coordination models enable agents to:
- **Share information** about market conditions
- **Coordinate actions** without explicit communication
- **Improve system efficiency** through implicit cooperation
- **Adapt to grid conditions** and resource availability


In [ ]:
# Example: Coordination model usage
# from src.coordination.implicit_cooperation import ImplicitCooperationModel

# Create implicit coordination model
# coordination_model = ImplicitCooperationModel(
#     agents=agents,
#     grid_capacity=100.0,
#     price_sensitivity=0.2,
#     congestion_threshold=0.8,
#     prediction_window=10,
#     privacy_level=0.7
# )

# Get coordination signals for agents
# signals = coordination_model.get_coordination_signals(agent_id, market_state)

# Calculate coordination metrics
# metrics = coordination_model.calculate_coordination_metrics(market_state, trades)
# print("Coordination Score:", metrics.coordination_score)

print("🤝 Coordination models enable implicit cooperation between agents.")
print("💡 Key features:")
print("   - Price trend signals")
print("   - Grid congestion awareness")
print("   - Resource availability tracking")
print("   - Coordination effectiveness metrics")


## 8. 📊 Analysis and Evaluation

### 📈 Metrics and Performance Evaluation

Metrics are used to evaluate the performance of the DERL-LEM framework. Key metrics include:

- **Market Price** 💰: The average price of energy traded in the market
- **Market Volume** ⚡: The total amount of energy traded in the market
- **Total Rewards** 🎯: The sum of rewards received by all agents
- **Coordination Score** 🤝: Measure of implicit cooperation effectiveness
- **Grid Stability** ⚖️: Measures of grid balance and constraint satisfaction

### 📉 Visualization

Visualization tools help researchers understand the dynamics of the market and the behavior of agents:

- **Market Price Trends**: Line graphs showing price evolution over time
- **Market Volume Trends**: Trading volume patterns
- **Agent Rewards**: Individual and cumulative reward tracking
- **Coordination Metrics**: Effectiveness of implicit cooperation
- **Grid State Visualization**: Network topology and constraint visualization


## 9. 🚀 Advanced Scenarios

### 🧪 Scenario Testing

Scenario testing involves simulating different market conditions to see how the framework performs under various scenarios:

- **Price Spikes** 📈: Simulating sudden price changes
- **Demand Changes** 📉: Simulating changes in energy demand
- **Grid Constraints** ⚠️: Simulating the impact of physical grid constraints
- **Agent Heterogeneity** 👥: Different agent types and capabilities
- **Battery Coordination** 🔋: Storage deployment strategies

### 🔬 Ablation Studies

Ablation studies involve removing different components to see their impact:

- **Removing Market Mechanisms**: Testing mechanism effectiveness
- **Removing Coordination Models**: Understanding coordination value
- **Removing Analysis Tools**: Impact on system understanding


## 10. 🤝 Preference-Based Matching

In local energy markets, agents may have preferences for specific trading partners based on various factors like physical proximity, complementary generation patterns, or past successful interactions.

### 🎯 Setting Up Preference-Based Matching

The DERL-LEM framework provides powerful capabilities for setting up and analyzing preference-based matching between agents.

**Available Preference Strategies:**

| Strategy        | Description                               | Best For                       |
| --------------- | ----------------------------------------- | ------------------------------ |
| **NONE**          | No preferences, purely price-based        | Traditional market simulations |
| **PROXIMITY**     | Prefer closest agents physically          | Minimizing transmission losses |
| **COMPLEMENTARY** | Prefer agents with opposite energy needs  | Balancing supply/demand        |
| **REPUTATION**    | Prefer agents with high reputation scores | Trust-based simulations        |
| **HISTORICAL**    | Prefer agents with past successful trades | Learning-based scenarios       |
| **RANDOM**        | Random preference selection               | Baseline comparisons           |
| **REINFORCEMENT** | Learn preferences through RL              | Advanced scenarios             |

**When preference-based matching is enabled:**
1. Each agent selects a preferred trading partner using its strategy
2. The market considers both price compatibility and preferences during matching
3. The `preference_weight` parameter controls the balance between preferences and price


In [ ]:
# Example: Preference-based matching setup
# market_config = MarketConfig(
#     min_price=0.05,
#     max_price=0.5,
#     enable_partner_preference=True,  # Enable preference-based matching
#     # ... other config parameters
# )

print("🤝 Preference-based matching allows agents to prefer specific trading partners.")
print("💡 Benefits:")
print("   - Reduced transmission losses (proximity-based)")
print("   - Better supply-demand matching (complementary)")
print("   - Trust-based trading (reputation)")
print("   - Learning from past interactions (historical)")


## 11. 🔬 Advanced Scientific Analysis

### 📊 Computing and Interpreting Scientific Metrics

The framework provides comprehensive metrics for scientific evaluation:

**Economic Efficiency Metrics:**
- Market efficiency scores
- Social welfare calculations
- Price discovery effectiveness
- Trade volume analysis

**Grid Stability Metrics:**
- Grid balance deviation
- Constraint violation tracking
- Voltage stability indicators
- Transmission loss analysis

**Coordination Metrics:**
- Implicit cooperation effectiveness
- Coordination score evolution
- Network structure analysis
- Preference formation patterns


In [ ]:
# Example: Scientific metrics computation
# from src.analytics.metrics import compute_economic_efficiency, compute_grid_stability

# After running a simulation and collecting:
# trades = list of Trade objects
# prices = list of market clearing prices
# grid_states = list of grid state variables

# efficiency_metrics = compute_economic_efficiency(trades, prices)
# stability_metrics = compute_grid_stability(grid_states, trades)

# print("Economic Efficiency Metrics:", efficiency_metrics)
# print("Grid Stability Metrics:", stability_metrics)

print("📊 Scientific metrics provide quantitative evidence for:")
print("   - Market efficiency and grid stability")
print("   - Coordination effectiveness")
print("   - System performance improvements")
print("   - Research claims validation")


## 12. ⚡ Quickstart Guide

### 🚀 Quick Start Example

Get up and running in minutes with this minimal example:


In [ ]:
# Quickstart: Minimal working example
from src.environment.lem import LocalEnergyMarket
from src.market.matching import MarketConfig, ClearingMechanism

# Minimal configuration
market_config = MarketConfig(
    min_price=0.05,
    max_price=0.5,
    min_quantity=0.0,
    max_quantity=100.0,
    price_mechanism=ClearingMechanism.AVERAGE
)

env_config = {
    "max_steps": 24,    # 24 hours
    "num_agents": 3,    # 3 agents
    "seed": 42,
    "market_config": market_config
}

# Create and run environment
env = LocalEnergyMarket(env_config=env_config)
obs, info = env.reset(seed=42)

# Run a few steps
for step in range(3):
    actions = {agent_id: env.action_space[agent_id].sample() for agent_id in env.agents}
    obs, rewards, terminated, truncated, info = env.step(actions)
    print(f"Step {step + 1}: Price=${info.get('market_price', 0):.4f}, Volume={info.get('market_volume', 0):.2f}")

print("\n✅ Quickstart complete!")


## 13. 🔄 Full Pipeline Examples

### 📦 Complete Workflow

Here's a complete workflow example showing the full pipeline from setup to analysis:


In [ ]:
# Full pipeline example
# This demonstrates the complete workflow: setup → simulation → analysis

print("🔄 Full Pipeline Workflow:")
print("=" * 60)
print("1. ⚙️  Configuration: Set up market, agents, grid")
print("2. 🎮 Simulation: Run environment steps")
print("3. 📊 Analysis: Calculate metrics and evaluate performance")
print("4. 📈 Visualization: Create plots and reports")
print("5. 💾 Storage: Save results for later analysis")
print("\n💡 See the 'src.ipynb' notebook for detailed step-by-step examples.")


### 🔁 Reproducibility and Experiment Management

**Best Practices:**

1. **Set Seeds**: Use consistent random seeds for reproducibility
   ```python
   import numpy as np
   np.random.seed(42)
   ```

2. **Save Configurations**: Store experiment configurations for later reference
   ```python
   from src.environment.io import EnvConfigHandler
   EnvConfigHandler().save(env_config, storage_path, "experiment_name")
   ```

3. **Version Control**: Track code versions and dependencies

4. **Document Parameters**: Keep detailed notes of all configuration choices


## 14. 🔧 Extending the Codebase

### ➕ How to Add New Components

**Add a new agent:**
- Inherit from `DERAgent`
- Override `act()` or `learn()` methods
- Implement custom decision-making logic

**Add a new market mechanism:**
- Modify or extend `LocalEnergyMarket`
- Implement a new class in `src/environment/`
- Define matching and pricing logic

**Add a new metric:**
- Implement in `src/analytics/metrics.py`
- Import and use in your pipeline
- Add visualization if needed

**Add a new coordination model:**
- Create in `src/coordination/`
- Implement signal generation logic
- Integrate with environment


## 15. ❓ Troubleshooting and FAQ

### 🔍 Common Issues and Solutions

**❌ Simulation runs but no trades:**
- ✅ Check agent action logic
- ✅ Ensure agents submit valid bids/offers
- ✅ Verify price ranges are compatible
- ✅ Check market mechanism configuration

**❌ Results not reproducible:**
- ✅ Ensure all seeds are set (NumPy, random, environment)
- ✅ Verify configurations are saved/loaded correctly
- ✅ Check for non-deterministic operations

**❌ Visualization errors:**
- ✅ Verify all plotting libraries are installed
- ✅ Check that metric data is well-formed
- ✅ Ensure data types are correct (lists, arrays, etc.)

**❌ Import errors:**
- ✅ Verify Python path is set correctly
- ✅ Check that all dependencies are installed
- ✅ Ensure you're in the correct directory
- ✅ Verify environment is activated

**❌ Performance issues:**
- ✅ Reduce number of agents for testing
- ✅ Decrease simulation length
- ✅ Use smaller grid networks
- ✅ Optimize batch processing


---

## 📝 Summary

This guide has walked you through:

1. ✅ **Installation and Setup** - Getting the framework ready
2. ✅ **Basic Simulation** - Running your first simulations
3. ✅ **Agent Configuration** - Working with different agent types
4. ✅ **Market Mechanisms** - Understanding trading mechanisms
5. ✅ **Coordination Models** - Implementing implicit cooperation
6. ✅ **Analysis and Evaluation** - Measuring performance
7. ✅ **Advanced Features** - Preference matching and scientific analysis

### 🎯 Next Steps

- 📚 **Explore case studies** in the `cases/` directory for detailed examples
- 🔬 **Read the research papers** for theoretical foundations
- 💻 **Check the source code** for implementation details
- 🤝 **Join the community** for discussions and support

### 📚 Additional Resources

- **Main Notebook**: See `notebooks/src.ipynb` for detailed step-by-step examples
- **Case Studies**: Explore `cases/` directory for advanced scenarios
- **Documentation**: Check `docs/` for detailed API documentation
- **Examples**: Review `examples/` for more code samples

---

**🚀 Happy coding! Start with the basic simulation and gradually explore more advanced features.**
